In [1]:
# syft absolute
import syft as sy

kj/filesystem-disk-unix.c++:1703: warning: PWD environment variable doesn't match current directory; pwd = /Users/koen/workspace/pysyft/notebooks


In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    # node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/q1/ryq93kwj055dlbpngxv1c7z40000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=f3846b1011624b31a33d93f9f46b122b in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")

Exception: No common communication protocol found between the client and the server.

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tm

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'

[Errno 2] No such file or directory: '/tm

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tm

[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/QueueItem.lock'
[Errno 2] No such file or directory: '/tmp/sherlock/SyftWorker.lock'
[Errno 2] No such file or directory: '/tm

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: High side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=private_low,
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

  0%|                                                                                                                 | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.92it/s]


# Make Requests

In [7]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(data) -> float:
    print("Computing mean...")
    return data.mean()


# compute_mean(data=data_low.mock)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

In [8]:
res = client_low_ds.code.request_code_execution(compute_mean)

In [9]:
res

syft.service.request.request.Request

In [10]:
node_low.python_node.get_service("usercodeservice").stash.partition.data

{<UID: 84e746712db04cdd98d4b72a85acfaaf>: syft.service.code.user_code.UserCode}

In [11]:
client_low_ds.code[0].status

{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}

In [12]:
# client_low.requests[0].accept_by_depositing_result(10.0)
client_low.requests[0].approve()

Approving request for domain test_l


SyftSuccess: Request 86c350bb865d420bacf13ba674f03690 changes applied

In [13]:
data = client_low_ds.datasets[0].assets[0]

job = client_low_ds.code.compute_mean(data=data, blocking=False)

job

```python
class Job:
    id: UID = 4344334414a24741a0c59f701181854d
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [14]:
job.wait()

override False
output_history [] 290152bb9cd74ea3971f7219d7b1dae6


29/02/24 15:31:39 FUNCTION LOG (4344334414a24741a0c59f701181854d): Computing mean...


```python
Pointer
```
7.0

In [15]:
client_low_ds.code.compute_mean(data=data).get()

override False
output_history [syft.service.output.output_service.ExecutionOutput] 290152bb9cd74ea3971f7219d7b1dae6
output_history [syft.service.output.output_service.ExecutionOutput] 290152bb9cd74ea3971f7219d7b1dae6


7.0

In [16]:
client_low_ds.code[0].output_policy

```python
class OutputPolicyExecuteOnce:
  id: str = 290152bb9cd74ea3971f7219d7b1dae6

```

In [17]:
node_low.python_node.get_service("outputservice").stash.partition.data

{<UID: 819f1ea4c95e471996886e0e350db4d1>: syft.service.output.output_service.ExecutionOutput}

In [18]:
node_low.python_node.get_service("outputservice").stash.partition.permissions

{<UID: 819f1ea4c95e471996886e0e350db4d1>: {'779c601cdb23dad8277d90890355c9d508d6f16dcb9639287b49c15acf8392e9_READ', '779c601cdb23dad8277d90890355c9d508d6f16dcb9639287b49c15acf8392e9_OWNER', '779c601cdb23dad8277d90890355c9d508d6f16dcb9639287b49c15acf8392e9_WRITE', '779c601cdb23dad8277d90890355c9d508d6f16dcb9639287b49c15acf8392e9_EXECUTE'}}

In [20]:
client_low_ds.verify_key

779c601cdb23dad8277d90890355c9d508d6f16dcb9639287b49c15acf8392e9

In [21]:
uid = client_low_ds.code[0].output_history[0].output_policy_id
client_low_ds.output.get_by_output_policy_id(uid)

[syft.service.output.output_service.ExecutionOutput]

In [22]:
client_low.sync.get_state()

syft.service.sync.sync_state.SyncState